# Análisis de Retención de Clientes (CHURN)


----

## Contexto del Negocio

El banco ficticio Monopoly, con años de operación en Chile, ha sido recientemente adquirido por una entidad extranjera: el banco Dormammu.

Como parte del proceso de integración, Dormammu ha solicitado un análisis exhaustivo de la base de datos de clientes de Monopoly —que contiene 574 variables y 51.124 registros— con el objetivo de:

* Comprender el comportamiento financiero de los clientes actuales.
* Identificar patrones de uso en productos como cuentas corrientes y tarjetas de crédito.
* Detectar oportunidades para mejorar los servicios existentes y aumentar la rentabilidad.
* Evaluar estrategias para retener clientes actuales y atraer nuevos usuarios hacia los productos del banco fusionado.

Este análisis será clave para diseñar políticas comerciales, campañas de fidelización, y planes de producto más alineados con los perfiles y necesidades detectadas en la base de clientes.

**Objetivo del Análisis**:

**Retención de Clientes**: Desarrollar un modelo predictivo que permita identificar clientes con riesgo de abandonar el banco. El objetivo es facilitar la creación de campañas de retención proactivas y focalizadas, basadas en patrones de comportamiento reales.

---

### Importaciones

In [ ]:
#importaciones de librerías
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import joblib
from seaborn import load_dataset
from seaborn import kdeplot
from seaborn import lmplot
from seaborn import boxplot

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from scipy.sparse import hstack

### Configuración del Dataset

In [ ]:
# Descargar dataset
path = kagglehub.dataset_download("nadiaarellanog/base-clientes-monopoly")

print("Ruta:", path)

In [ ]:
monopoly = pd.read_csv(path + "/Base_clientes_Monopoly.csv", delimiter='\t')

In [ ]:
# Mostrar las primeras filas del DataFrame para verificar la carga del archivo
print(monopoly.head())

---

### Comprensión de los Datos

El conjunto de datos proporcionado por el banco Monopoly (ahora parte de Dormammu) contiene información detallada de más de 51.000 clientes y 574 variables, abarcando:

* Datos demográficos
* Información sobre productos financieros (cuentas, tarjetas, créditos)
* Actividad transaccional mensual (volumen y frecuencia)
* Indicadores de comportamiento y engagement

Este dataset permitirá construir una visión rica y detallada de cada cliente, así como modelar su propensión a continuar utilizando los servicios del banco.


### Objetivos de esta Fase

Durante la comprensión de los datos se busca:

* Entender la estructura y significado de las variables disponibles, incluyendo unidades, escalas y formatos.
* Detectar problemas de calidad de datos: valores faltantes, formatos inconsistentes, tipos incorrectos o valores atípicos.
* Identificar variables relevantes para el modelo de retención, basándose en su naturaleza, correlación con el target y distribución.


### Descripción de Variables Relevantes

A continuación se resumen algunas de las variables clave agrupadas por tipo, destacando su utilidad en el análisis:

🔹 Variables Demográficas
* Región: Zona geográfica de residencia del cliente. Útil para segmentación geográfica.
* Edad: Factor clave en el comportamiento financiero y tipo de productos utilizados.
* Sexo: Potencialmente relevante para personalización de estrategias.
* Renta: Indicador del poder adquisitivo del cliente, relacionado con su volumen de uso de productos.

🔹 Productos Financieros
* Ctacte: Tenencia de cuenta corriente. Asociada a clientes activos.
* Hipotecario: Presencia de crédito hipotecario. Generalmente asociado a mayor compromiso con el banco.
* Debito / TC: Indican si el cliente posee tarjetas de débito o crédito, respectivamente.

🔹 Actividad Transaccional
* Txs_T12: Número de transacciones en tarjetas de crédito durante el mes más reciente.
* CUPO_L1 / L2 / MX: Límites de crédito en compras nacionales, avances y compras internacionales.
* FlgAct_T12: Indicador de actividad reciente en tarjeta de crédito.


### Problemas Detectados en el Análisis Inicial

Durante la exploración preliminar se identificaron los siguientes desafíos:

* Valores nulos en variables clave como Renta, Región y Sexo.
* Tipos inconsistentes: Algunas variables numéricas aparecen como tipo object por uso de comas en lugar de puntos decimales o símbolos extraños.
* Outliers: En variables como Txs_T12 y FacAN_T12 se observan valores atípicos que podrían afectar el entrenamiento del modelo.

Estos problemas se abordarán en la etapa de preparación de los datos, mediante imputación, conversión de tipos y tratamiento de valores extremos según sea necesario.


### Exploraciones Iniciales Realizadas

Se realizaron algunas visualizaciones para comprender mejor la distribución de los datos:

* Subsegmentos de Clientes: Permite observar agrupaciones internas según nivel de productos, ingresos u otros factores.
* Distribución Regional: Ofrece perspectiva sobre la presencia geográfica de los clientes.
* Género: Ayuda a identificar sesgos o necesidades de segmentación.

---

Se carga el conjunto de datos original monopoly y se convierte en un DataFrame de Pandas para facilitar su manipulación y análisis:

In [26]:
df = pd.DataFrame(monopoly)

Esta conversión permite trabajar con una estructura tabular estándar en Python, ideal para aplicar operaciones de limpieza, transformación y exploración utilizando herramientas del ecosistema científico como pandas, numpy y scikit-learn.

In [ ]:
#Cantidad de filas(registros) x cantidad de columnas(variables)
print(df.shape)

Este comando muestra las dimensiones del DataFrame, es decir:

* **Filas** (registros): 51,124 clientes
* **Columnas** (variables): 574 características asociadas a cada cliente

Esta información es clave para dimensionar el volumen de datos con el que se trabajará durante las etapas de análisis y modelado.



In [ ]:
df.head()

In [ ]:
# Registro aleatorio
df.sample()

In [ ]:
# Genera el gráfico de barras
plt.figure(figsize=(12, 6))
conteo_subsegmento = df['Subsegmento'].value_counts()
ax = conteo_subsegmento.plot.bar(rot=45, color='orange', width=0.8)

# Etiquetas de texto para cada barra
for i, v in enumerate(conteo_subsegmento):
    ax.text(i, v + 5, str(v), ha='center', va='bottom')

plt.xlabel("Subsegmento")
plt.ylabel("Frecuencia")
plt.title("Distribución de Subsegmento")

plt.show()

El gráfico de barras muestra que el subsegmento "170" es el más frecuente en la base de datos, lo que indica que esta categoría domina en esta variable.

In [ ]:
# Genera el gráfico de barras
plt.figure(figsize=(17, 6))
conteo_region = df['Region'].value_counts()
ax = conteo_region.plot.bar(rot=0, color='purple', width=0.4)

# Etiquetas de texto para cada barra
for i, v in enumerate(conteo_region):
    ax.text(i, v + 5, str(v), ha='center', va='bottom')

plt.xlabel("Region")
plt.ylabel("Frecuencia")
plt.title("Distribución de Región")

plt.show()

La variable `Region` presenta una fuerte concentración en la categoría 13, lo que coincide con su moda. No obstante, se observan inconsistencias en el formato de algunas regiones, como `7` y `7.0`, que representan el mismo valor pero en diferentes tipos. Estas inconsistencias serán tratadas en la fase de limpieza de datos.

In [ ]:
# Genera el gráfico de barras
plt.figure(figsize=(17, 6))
conteo_sexo = df['Sexo'].value_counts()
ax = conteo_sexo.plot.bar(rot=0, color='green', width=0.4)

# Etiquetas de texto para cada barra
for i, v in enumerate(conteo_sexo):
    ax.text(i, v + 5, str(v), ha='center', va='bottom')

plt.xlabel("Sexo")
plt.ylabel("Frecuencia")
plt.title("Distribución de Género: Hombres vs. Mujeres")

plt.show()

El gráfico de barras muestra que la categoría masculina es la más representada en el conjunto de datos, indicando una mayor proporción de hombres que mujeres en la muestra.

In [ ]:
# Eliminar valores nulos de la columna que se va a graficar
data_to_plot = df['Txs_T12'].dropna()

# Crear el histograma
plt.figure(figsize=(12, 7))
plt.hist(data_to_plot, bins=30, alpha=0.7, color='skyblue', edgecolor='black')

# Añadir título y etiquetas
plt.title('Histograma de Transacciones (Txs_T12)', fontsize=16)
plt.xlabel('Valor de Transacciones', fontsize=14)
plt.ylabel('Frecuencia', fontsize=14)

# Añadir líneas de referencia
mean_value = data_to_plot.mean()
plt.axvline(mean_value, color='red', linestyle='dashed', linewidth=1, label=f'Media: {mean_value:.2f}')

# Añadir leyenda
plt.legend()

# Añadir cuadrícula
plt.grid(axis='y', alpha=0.75)

# Mostrar el histograma
plt.tight_layout()  # Ajustar para que todo se muestre correctamente
plt.show()

El histograma muestra la distribución del número de transacciones mensuales en tarjeta de crédito (`Txs_T12`). Se observa una alta concentración de clientes con pocos movimientos, lo que indica un uso limitado del producto por la mayoría de la base.

Una línea discontinua roja representa la media (**~2.59 transacciones**), visiblemente desplazada hacia la derecha de la mayoría de los datos, lo que evidencia un sesgo fuerte hacia valores bajos.

**Observaciones clave**

* **Distribución sesgada**: La variable presenta una distribución fuertemente sesgada a la derecha (right-skewed), con la mayoría de los valores cercanos a cero.
* **Media baja**: Aunque el promedio se sitúa en 2.59, este valor está influido por unos pocos clientes con volúmenes transaccionales altos.
* **Baja actividad general**: La gran mayoría de los clientes realiza pocas transacciones mensuales, lo que puede implicar baja utilización del producto o inactividad.



In [ ]:
# Filtrar valores nulos en la columna Txs_T12
data_to_plot = df['Txs_T12'].dropna()

# Crear el diagrama de caja (boxplot)
plt.figure(figsize=(10, 6))
plt.boxplot(data_to_plot, vert=False, patch_artist=True,
            boxprops=dict(facecolor='lightblue'), whis=10)  # Valor whis ajustado

# Añadir título y etiquetas de los ejes
plt.title('Diagrama de Caja de Transacciones (Txs_T12)', fontsize=16)
plt.xlabel('Valor de Transacciones', fontsize=14)

# Agregar la curva de densidad (KDE plot) con transparencia
sns.kdeplot(data_to_plot, color='blue', bw_adjust=0.5, alpha=0.5)  # Ajuste de suavizado y transparencia

# Mostrar el diagrama de caja y configurar la cuadrícula
plt.grid(axis='x', alpha=0.75)
plt.tight_layout()  # Asegurar que todos los elementos se muestren correctamente
plt.show()


El gráfico combina un diagrama de caja con una curva de densidad (KDE) para representar la distribución de transacciones mensuales (`Txs_T12`). Ambos elementos revelan una alta concentración de valores cercanos a cero, con una caída abrupta a medida que aumentan.

El diagrama de caja muestra una dispersión estrecha en el rango bajo, con outliers significativos en el extremo derecho, lo que indica la presencia de clientes con actividad inusualmente alta. Los bigotes se extienden hasta aproximadamente 50 transacciones, mientras que el KDE evidencia un fuerte sesgo a la derecha, con un único pico dominante.

**Observaciones clave**

* **Alta concentración en valores bajos**: La mayoría de los clientes realiza muy pocas transacciones mensuales.
* **Outliers relevantes**: Existen casos aislados con volúmenes considerablemente más altos que el promedio, lo cual influye en la media.
* **Distribución asimétrica**: La forma de la curva y el diagrama de caja evidencian una distribución fuertemente sesgada (right-skewed), algo típico en datos financieros de uso de productos.

---



## Preparación de los Datos

En esta fase, se limpia y transforma el dataset para asegurar que sea consistente, completo y adecuado para el modelado. Dado que se detectaron valores nulos, tipos inconsistentes y outliers, se aplicarán transformaciones específicas para garantizar la calidad de los datos.


### Objetivos de esta Etapa
Los principales objetivos de esta fase incluyen:

A. **Limpieza de Nulos**: Identificar y tratar valores nulos, tipos incorrectos y registros anómalos.

B. **Conversión y Transformación**: Asegurar que las variables estén correctamente tipadas y representadas.

C. **Tratamiento de Outliers**: Limpiar los valores extremos detectados.

D. **Normalización/Estandarización**: Escalar variables financieras para asegurar comparabilidad en el modelado.

### A. Limpieza de Nulos
1. **Variables demográficas**: Sexo y Región contienen valores faltantes. Dado su potencial valor analítico, se imputarán utilizando la moda o el valor más frecuente dentro de subgrupos.
2. **Variable financiera – Renta**: Dado su impacto en el comportamiento del cliente, se imputará usando la mediana por subsegmento, para preservar relaciones contextuales.
3. Se incluirá un heatmap de nulos y un gráfico de barras para identificar las columnas más afectadas.

### B. Conversión y Transformación
1. Variables mal tipadas como object (ej. Renta, CUPO_L1, etc.) serán convertidas a float, eliminando símbolos o formatos inválidos.
2. Región será convertida a tipo categórico, ya que representa una ubicación y no una magnitud continua.

### C. Tratamiento de Outliers
Se identificaron valores extremos en variables como Txs_T12 y FacAN_T12. Para ello:

1. Se usarán boxplots y curvas KDE para detectar y visualizar estas anomalías.
2. El tratamiento aplicará reglas basadas en el rango intercuartílico (IQR) o transformaciones logarítmicas, según el impacto observado.

### D. Normalización y Estandarización
1. Estandarización (z-score) se aplicará a variables financieras como Renta, CUPO_L1, CUPO_L2 y CUPO_MX.
2. Normalización se considerará para variables transaccionales si presentan asimetría fuerte.

Estas técnicas aseguran que las variables contribuyan equitativamente en los modelos, sin que su escala domine el aprendizaje.

### Preparación del Target para el Modelado
El target será la suma anual de transacciones con tarjetas (Txs_T01 a Txs_T12), una métrica que representa la actividad del cliente durante un año.

Pasos:
1. **Cálculo**: Se sumarán las columnas mensuales para crear total_transacciones_anual.
2. **Transformación**: Dado el sesgo hacia valores bajos, se aplicará una transformación logarítmica (log1p) para mejorar la distribución y el ajuste en modelos de regresión.

### Resultado Final
Esta etapa deja el dataset limpio, tipado correctamente y escalado, listo para aplicar modelos de regresión o clasificación. La calidad de esta preparación es clave para generar modelos robustos y relevantes para las necesidades estratégicas del banco Dormammu.

---

In [36]:
# Selecciona todas las columnas mensuales de transacciones
transacciones_mensuales = [f'Txs_T{i:02}' for i in range(1, 13)]

# Verifica que las columnas existan en el DataFrame
transacciones_mensuales = [col for col in transacciones_mensuales if col in df.columns]

# Calcula el total anual de transacciones
df['total_transacciones_anual'] = df[transacciones_mensuales].sum(axis=1)
target_regresion = df['total_transacciones_anual']


In [37]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Análisis clientes
# Utiliza describe() para obtener estadísticas de variables categóricas y numéricas
df_describe = df.iloc[:, 0:21].describe(include='all')

# Filtra solo las columnas count, unique, top, freq
df_filtered = df_describe.loc[['count', 'unique', 'top', 'freq']]

# Muestra solo las estadísticas seleccionadas
print(df_filtered)

             Id  Subsegmento   Sexo   Region     Edad     Renta  Antiguedad  Internauta  Adicional  Dualidad  Monoproducto   Ctacte  Consumo  Hipotecario   Debito  CambioPin  Cuentas       TC  CUPO_L1  CUPO_L2  CUPO_MX
count   51124.0      51124.0  51123  51071.0  51124.0   37759.0     51124.0     51124.0    51124.0   51124.0       51124.0  51124.0  51124.0      51124.0  51124.0    31736.0  51124.0  51124.0  51124.0  51124.0  51124.0
unique      NaN          NaN      2      NaN      NaN   34276.0         NaN         NaN        NaN       NaN           NaN      NaN      NaN          NaN      NaN        NaN      NaN      NaN      NaN   4591.0   3522.0
top         NaN          NaN      H      NaN      NaN  400000.0         NaN         NaN        NaN       NaN           NaN      NaN      NaN          NaN      NaN        NaN      NaN      NaN      NaN      1.0   1000.0
freq        NaN          NaN  27410      NaN      NaN     205.0         NaN         NaN        NaN       NaN           NaN  

Se inspeccionaron las estadísticas básicas de un subconjunto de variables utilizando df.describe(include='all'). A continuación se destacan observaciones clave por tipo de variable:

🔹 Identificadores y Categóricas
* **Id**: Completo (51.124 valores), actúa como identificador único.
* **Subsegmento**: Completo, pero no se muestran métricas adicionales al tratarse de una variable categórica numérica.
* **Sexo**: Casi completa (1 valor nulo). Dos categorías, con predominio masculino (H, ~27.410 casos).
* **Región**: Presenta valores faltantes (~53 ausentes). Deberá convertirse a categórica y limpiarse por formato inconsistente (ej. "7" vs "7.0").

🔹 Variables Numéricas
* **Edad, Antigüedad**: Completas y numéricas continuas, sin issues aparentes.
* **Renta**: Incompleta (~26% nulos). Altamente variable (34.276 valores únicos). El valor más frecuente es 400.000 (205 registros), lo que podría reflejar un default artificial o segmento común.

🔹 Variables Indicadoras (Binarias)
* Variables como **Internauta, Adicional, Dualidad, Monoproducto, Ctacte, Consumo, Hipotecario, Debito** están completas y parecen binarias (0/1). Su análisis de frecuencias será útil para evaluar activación o tenencia de productos.

🔹 Variables con Alta Ausencia
* **CambioPin**: ~38% de valores nulos. Requiere decisión sobre imputación o eliminación, según su relevancia en el modelo.

🔹 Límites de Crédito
* **CUPO_L1 / L2 / MX**: Completos, con alta cardinalidad. Valores frecuentes como 1000.0 o 1.0 podrían corresponder a cupos mínimos estandarizados o defaults del sistema.


In [ ]:
#Análisis de movimientos durante el mes de Diciembre
df.iloc[::, list(range(21, 67)) + [573]].describe()

In [39]:
# Crea una copia del DataFrame para no modificar el original en las siguientes ejecuciones
df_copia = df.copy()

In [40]:
# Mostrar la media
print(df.iloc[:, 0:21].dtypes)

Id                int64
Subsegmento       int64
Sexo             object
Region          float64
Edad              int64
Renta            object
Antiguedad        int64
Internauta        int64
Adicional         int64
Dualidad          int64
Monoproducto      int64
Ctacte            int64
Consumo           int64
Hipotecario       int64
Debito            int64
CambioPin       float64
Cuentas           int64
TC                int64
CUPO_L1           int64
CUPO_L2          object
CUPO_MX          object
dtype: object


Se inspeccionaron los tipos de las primeras columnas del dataset para identificar inconsistencias relevantes:

* **Categóricas mal tipadas**: Variables como Renta, CUPO_L2, CUPO_MX están en formato object, lo que sugiere problemas de formato (p. ej., uso de comas decimales o símbolos no numéricos). Se transformarán a float tras limpieza.
* **Variables binarias**: Columnas como Internauta, Ctacte, Hipotecario, entre otras, están correctamente como int, aunque podrían representarse como bool para mejorar la semántica.
* **Región**: Está en formato float64, pero representa una categoría geográfica. Será convertida a tipo category para reflejar su naturaleza.

Este diagnóstico guiará la conversión y estandarización de tipos durante la limpieza.


In [ ]:
# Seleccionar solo las columnas numéricas
df_numerico = df.iloc[:, 0:21].select_dtypes(include=[np.number])

# Calcular la mediana de las columnas numéricas
mediana = df_numerico.median()

# Mostrar la mediana
print(mediana)

In [ ]:
subset_df = df.iloc[:, :21]
moda_result = subset_df.mode()
print(moda_result.iloc[0])

Se analizaron las medianas y modas de las variables clave para identificar tendencias centrales en el comportamiento y perfil de los clientes:

🔹Perfil Demográfico y General
* **Edad**: Mediana de 35 años, con moda en 27 → clientela relativamente joven.
* **Antigüedad**: Mediana de 25 meses, moda de 10 meses → predominan clientes nuevos.
* **Región**: Moda en 13.0, lo que indica alta concentración geográfica en esa zona (probablemente Santiago).
* **Sexo**: Moda “H” → mayoría de los clientes son hombres.

🔹Productos y Servicios
* **Internauta, Débito, Ctacte, CambioPin**: Moda de 1, lo que sugiere alto uso de servicios digitales y productos básicos.
* **Créditos (Consumo, Hipotecario), Adicionales, Dualidad**: Moda de 0 → baja adopción de productos avanzados.

🔹Ingresos y Cupos
* **Renta**:
  * Mediana no mostrada por estar incompleta (~26% de nulos).
  * Moda en 400.000 → posible valor común o default.
* **CUPO_L1**: Mediana en 800.000, moda en 200.000 → límites estandarizados.
* **CUPO_MX**: Moda en 1.000, indicando cupos internacionales bajos.

🔹Tarjetas de Crédito
* **TC (número de tarjetas)**: Mediana de 2, moda de 1 → mayoría con una sola tarjeta, pero con una fracción relevante que posee más.

🔹Conclusión

El banco mantiene una base joven, mayoritariamente masculina y reciente, con acceso preferente a productos digitales y básicos. La baja adopción de productos financieros complejos sugiere oportunidades en venta cruzada (cross-sell) o estrategias de fidelización.

In [ ]:
df_copia.isnull().sum()

Se identificaron valores nulos en un subconjunto de variables:

| Variable | Nulos | Comentario                                                               |
| -------- | ----- | ------------------------------------------------------------------------ |
| Sexo     | 1     | Valor faltante puntual, puede imputarse con la moda                      |
| Region   | 53    | Faltantes más relevantes; requiere análisis antes de imputar o eliminar. |
| Otras    | 0     | Completas                                                                |

🔹Observaciones clave:
* La mayoría de las columnas críticas están completas.
* Las columnas con nulos tienen baja proporción sobre el total (51.124 registros), por lo que imputar con valores dominantes o eliminar esos pocos registros no tendría impacto significativo.
* En etapas posteriores se definirá si se imputan (Region, Sexo) o si se excluyen, según su relevancia en el modelado.


In [ ]:
# Conversión de Renta a numérico
df_copia['Renta'] = pd.to_numeric(df['Renta'], errors='coerce')

# Imputaciones
df_copia['Sexo'] = df_copia['Sexo'].fillna(df['Sexo'].mode()[0])
df_copia['Region'] = df_copia['Region'].fillna(df['Region'].mode()[0])
df_copia['Renta'] = df_copia.groupby('Subsegmento')['Renta'].transform(lambda x: x.fillna(x.median()))
df_copia['CambioPin'] = df_copia['CambioPin'].fillna(df['CambioPin'].median())

# Nota: Aunque actualmente no hay nulos en 'Renta' ni 'CambioPin',
# estas imputaciones están incluidas como medida preventiva para garantizar
# estabilidad si se aplicara el pipeline a nuevos datos.

Este bloque de código ejecuta la limpieza e imputación de datos faltantes en el DataFrame df_copia, con el objetivo de garantizar que las columnas críticas estén completas y correctamente tipadas. Esta etapa es esencial para evitar errores en etapas posteriores del análisis y asegurar la integridad del pipeline de modelado.


In [ ]:
# Convertir variables numéricas que están en formato object a float
# Primero, eliminar caracteres especiales que impidan la conversión

# Remover posibles símbolos de moneda y convertir a numérico
df_copia['Renta'] = pd.to_numeric(df['Renta'], errors='coerce')
df_copia['CUPO_L1'] = pd.to_numeric(df['CUPO_L1'], errors='coerce')
df_copia['CUPO_L2'] = pd.to_numeric(df['CUPO_L2'], errors='coerce')
df_copia['CUPO_MX'] = pd.to_numeric(df['CUPO_MX'], errors='coerce')

# Verificar la conversión
print(df[['Renta', 'CUPO_L1', 'CUPO_L2', 'CUPO_MX']].dtypes)

# Convertir `Region` a tipo categórico
df_copia['Region'] = df_copia['Region'].astype('category')


In [46]:
# Función para filtrar outliers utilizando el rango intercuartílico (IQR)
def remove_outliers(df, column):
    Q1 = df_copia[column].quantile(0.25)
    Q3 = df_copia[column].quantile(0.75)
    IQR = Q3 - Q1
    # Filtra los datos fuera del rango IQR
    filtered_df = df_copia[(df_copia[column] >= Q1 - 1.5 * IQR) & (df_copia[column] <= Q3 + 1.5 * IQR)]
    return filtered_df

# Aplicar la función en algunas de las variables financieras clave
df_copia = remove_outliers(df_copia, 'Renta')
df_copia = remove_outliers(df_copia, 'CUPO_L1')
df_copia = remove_outliers(df_copia, 'CUPO_L2')
df_copia = remove_outliers(df_copia, 'CUPO_MX')


In [ ]:
# Estandarización de variables financieras clave (para modelos de regresión lineal o logística)
scaler = StandardScaler()
df_copia[['Renta', 'CUPO_L1', 'CUPO_L2', 'CUPO_MX']] = scaler.fit_transform(df_copia[['Renta', 'CUPO_L1', 'CUPO_L2', 'CUPO_MX']])

# Normalización de las variables de transacciones (para modelos como K-Means)
transacciones_mensuales = [f'Txs_T{i:02}' for i in range(1, 13)]
scaler = MinMaxScaler()
df_copia[transacciones_mensuales] = scaler.fit_transform(df_copia[transacciones_mensuales])

# Revisar los resultados de la normalización y estandarización
print(df_copia[['Renta', 'CUPO_L1', 'CUPO_L2', 'CUPO_MX']].describe())
print(df_copia[transacciones_mensuales].describe())

In [ ]:
# Verificar el tipo de dato del target
print("Tipo de dato del target:", df_copia['total_transacciones_anual'].dtype)

# Visualizar la distribución del target con un histograma y un gráfico de densidad
plt.figure(figsize=(12, 6))

# Histograma
plt.subplot(1, 2, 1)
plt.hist(df_copia['total_transacciones_anual'].dropna(), bins=30, color='skyblue', edgecolor='black')
plt.title('Distribución del Target (Histograma)')
plt.xlabel('Total de Transacciones Anual')
plt.ylabel('Frecuencia')

# Gráfico de Densidad (KDE)
plt.subplot(1, 2, 2)
sns.kdeplot(df_copia['total_transacciones_anual'].dropna(), color='blue')
plt.title('Distribución del Target (Densidad)')
plt.xlabel('Total de Transacciones Anual')
plt.ylabel('Densidad')

plt.tight_layout()
plt.show()


🔹Análisis del Target
* **Tipo de Dato**:
  * El target, correspondiente al total anual de transacciones por cliente, está en formato float64. Este tipo de dato es adecuado para modelado de regresión, ya que representa una variable numérica continua.
* **Distribución del Target – Histograma**:
  * La mayoría de los clientes concentra sus transacciones anuales en valores bajos, próximos a cero.
  * La distribución presenta un sesgo positivo (cola a la derecha), con una frecuencia decreciente a medida que aumentan las transacciones. Pocos clientes superan las 100 transacciones anuales.
* **Distribución del Target – Curva de Densidad (KDE)**:
  * El KDE reafirma la concentración en valores bajos, con un pico marcado en la zona inicial de la escala y una larga cola hacia la derecha.
  * Este patrón sugiere una fuerte asimetría en la base de clientes: una mayoría con baja actividad y una minoría altamente activa.

🔹Conclusiones
* **Distribución Sesgada**:
  * El sesgo del target puede afectar la capacidad de los modelos de regresión para generalizar correctamente, especialmente si el objetivo es predecir con precisión los valores más altos (clientes altamente activos).
* **Transformación Recomendada**:
  * Aplicar una transformación logarítmica o raíz cuadrada podría reducir la asimetría y estabilizar la varianza, facilitando un aprendizaje más equilibrado por parte del modelo. Esta decisión dependerá de si se prioriza mejorar el ajuste general o capturar mejor los valores extremos.

In [ ]:
# Aplicar una transformación logarítmica al target para reducir el sesgo
df_copia['total_transacciones_anual_log'] = np.log1p(df_copia['total_transacciones_anual'])

# Visualizar la distribución del target transformado
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(df_copia['total_transacciones_anual_log'].dropna(), bins=30, color='skyblue', edgecolor='black')
plt.title('Distribución del Target Transformado (Histograma)')
plt.xlabel('Log(1 + Total de Transacciones Anual)')
plt.ylabel('Frecuencia')

plt.subplot(1, 2, 2)
sns.kdeplot(df_copia['total_transacciones_anual_log'].dropna(), color='blue')
plt.title('Distribución del Target Transformado (Densidad)')
plt.xlabel('Log(1 + Total de Transacciones Anual)')
plt.ylabel('Densidad')

plt.tight_layout()
plt.show()


🔹Evaluación de la Transformación Logarítmica del Target

La aplicación de una transformación logarítmica al target ha generado mejoras significativas en su distribución, lo que tiene implicancias directas sobre la calidad del modelado.
* **Distribución Más Simétrica**:
  * Tras la transformación, los valores del target se distribuyen de forma más equilibrada alrededor del centro. La forma de la curva se aproxima a una distribución normal, lo que favorece el uso de modelos que asumen simetría en los residuos.
* **Reducción del Sesgo**:
  * El sesgo positivo presente en los datos originales —concentración en valores bajos y una cola larga hacia la derecha— se ha mitigado. Esto reduce la influencia desproporcionada de los valores extremos sobre el ajuste del modelo.
* **Beneficios para el Modelado Predictivo**:
  * Una distribución más balanceada permite que los algoritmos de regresión capturen relaciones con mayor precisión a lo largo de todo el rango del target. Esto puede traducirse en:
    * Mejor capacidad de generalización.
    * Reducción del error en predicciones extremas.
    * Menor sensibilidad a outliers.

🔹Justificación del Uso de np.log1p para la Transformación del Target

Se utilizó np.log1p(x) en lugar de np.log(x) para transformar el target debido a las siguientes razones:
* **Manejo de ceros sin errores**:
  * A diferencia de np.log(x), que no admite valores iguales a cero (lo que generaría un error o -inf), np.log1p(x) calcula log(1 + x), permitiendo transformar valores cero de forma segura. Esto es especialmente relevante en contextos donde el target puede representar conteos o montos que incluyen ceros.
* **Estabilidad numérica**:
  * np.log1p ofrece mayor precisión para valores pequeños de x, evitando errores de redondeo o pérdida de información significativa que puede ocurrir con np.log(1 + x) de forma manual.
* **Conservación de la escala relativa**:
  * Aunque suaviza menos que una logaritmización más agresiva, np.log1p mantiene relaciones proporcionales importantes para modelos de regresión que requieren interpretar variaciones relativas.
* **Conclusión**:
  * El uso de np.log1p garantiza una transformación segura, estable y efectiva del target, facilitando una mejor distribución sin comprometer la integridad de los datos.

---

## Modelado

En esta etapa se implementan modelos de machine learning con el objetivo de identificar patrones en el comportamiento de los clientes y predecir su probabilidad de retención.

🔹Objetivo del Modelado

El propósito principal es construir un modelo supervisado capaz de clasificar a los clientes en función de su comportamiento financiero, específicamente en relación con su nivel de actividad transaccional anual (target binarizado). Este tipo de análisis es clave para anticipar abandono o desuso de productos financieros y orientar campañas de retención.

🔹Modelos Considerados

* **Regresión Logística**:
  * Modelo base de clasificación binaria, útil por su interpretabilidad y capacidad para manejar relaciones lineales entre variables independientes y la probabilidad de una clase (en este caso, clientes con alta o baja actividad).
* **Árboles de Decisión**:
  * Utilizados como alternativa más flexible, permiten capturar relaciones no lineales y generan reglas claras para interpretar la segmentación de clientes. Son especialmente útiles en entornos con múltiples variables categóricas y umbrales de decisión.
* **K-Means (Clustering)**:
  * Aplicado como método no supervisado para segmentar clientes en grupos según similitudes en sus patrones de uso. Este análisis complementa el modelado supervisado, ya que permite descubrir perfiles ocultos en la base de clientes, incluso sin etiquetas explícitas.

🔹Preparación de los Datos para el Modelado
* **División de Datos**:
  * El dataset se divide en un conjunto de entrenamiento (80%) y uno de prueba (20%) mediante train_test_split, garantizando que la evaluación del modelo se realice sobre datos no vistos.
* **Validación Cruzada**:
  * En el caso de modelos supervisados como la regresión logística, se emplea validación cruzada (k-fold) para evaluar la estabilidad del modelo y prevenir el sobreajuste. Esto permite obtener métricas más robustas en escenarios con alta varianza.
* **Estandarización y Codificación**:
  * Se construye un pipeline de preprocesamiento que transforma las variables numéricas (escaladas con StandardScaler) y categóricas (codificadas con OneHotEncoder), permitiendo que el modelo procese adecuadamente las distintas escalas y tipos de variables.


In [ ]:
# Selección de features y target (con transformación logarítmica)
X = df_copia.drop(columns=['total_transacciones_anual', 'total_transacciones_anual_log'])  # todas las columnas menos el target
y = df_copia['total_transacciones_anual_log']  # el target log-transformado

# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

🔹División de Variables y Separación de Datos

1. **Selección de Variables Predictoras (X) y Target (y)**
* Se define `X` como el subconjunto de variables independientes, excluyendo tanto el target original (`total_transacciones_anual`) como su transformación (`total_transacciones_anual_log`), para evitar fuga de información durante el entrenamiento.
* La variable objetivo y corresponde a `total_transacciones_anual_log`, una transformación logarítmica del total de transacciones anuales. Esta transformación permite suavizar la distribución del target y mejora el rendimiento en modelos sensibles al sesgo como la regresión lineal.

2. **División del Dataset en Entrenamiento y Prueba**
* Se utiliza `train_test_split` para dividir el dataset en un conjunto de entrenamiento (`80%`) y otro de prueba (`20%`).
* El parámetro `random_state=42` garantiza la reproducibilidad del experimento, asegurando que la partición de datos sea consistente a lo largo de ejecuciones sucesivas.

In [ ]:
# Identificar columnas que son de tipo `object`
object_columns = X.select_dtypes(include=['object']).columns

# Intentar convertir columnas numéricas con formato de coma decimal
for col in object_columns:
    try:
        # Reemplazar comas con puntos y convertir a float
        X[col] = X[col].str.replace(',', '.').astype(float)
        print(f"Columna {col} convertida a numérica con éxito.")
    except ValueError:
        print(f" /!\ Advertencia: Columna '{col}' no se pudo convertir a numérica, aún contiene valores no numéricos.")


🔹Conversión de Columnas Categóricas Mal Tipadas a Formato Numérico

Este bloque de código estandariza columnas originalmente clasificadas como object que, en realidad, contienen datos numéricos representados con comas decimales (formato común en regiones hispanohablantes).

* **Detección de Columnas Problemáticas**
  * Se identifican todas las columnas con tipo de dato `object`, ya que estas pueden contener valores numéricos mal interpretados como texto por el parser inicial (por ejemplo, `982,45`).
* **Normalización del Formato Numérico**
  * Para cada columna identificada, se reemplazan las comas por puntos (',' → '.') y se intenta convertir los valores a tipo `float`.
  * Las conversiones exitosas se notifican con un mensaje informativo; si la conversión falla (por ejemplo, por contener texto irreconciliable con una representación numérica), se genera una advertencia controlada.

Este paso es fundamental para evitar errores durante el preprocesamiento y asegurar que todas las variables numéricas estén en el formato adecuado para el análisis y modelado.

In [ ]:
# Detectar columnas categóricas en el DataFrame X
categorical_columns = X.select_dtypes(include=['object', 'category']).columns

# Identificar qué columnas pueden utilizar LabelEncoder y cuáles requieren OneHotEncoder
label_encode_cols = []
onehot_encode_cols = []

# Iterar sobre las columnas categóricas y contar las categorías únicas
for col in categorical_columns:
    unique_values = X[col].nunique()

    if unique_values == 2:
        # Si tiene solo dos categorías, se puede usar LabelEncoder
        label_encode_cols.append(col)
    else:
        # Si tiene más de dos categorías, se recomienda OneHotEncoder
        onehot_encode_cols.append(col)

# Mostrar resultados
print("Columnas para LabelEncoder (2 categorías):", label_encode_cols)
print("Columnas para OneHotEncoder (más de 2 categorías):", onehot_encode_cols)


🔹Clasificación de Variables Categóricas para Codificación

Este bloque organiza las columnas categóricas en función del tipo de codificación más apropiado, optimizando así su tratamiento antes del modelado.

* **Identificación de Columnas Categóricas**
  * Se seleccionan todas las columnas en `X` con tipo `object` o `category`, lo que usualmente representa variables con etiquetas no numéricas (por ejemplo, género, región, tipo de producto).
* **Segmentación según Número de Categorías**
  * Cada columna categórica se analiza según su cardinalidad (cantidad de valores únicos).
    * Si tiene exactamente dos categorías, se asigna a `label_encode_cols`, ya que puede representarse eficientemente con `LabelEncoder`.
    * Si tiene más de dos categorías, se asigna a `onehot_encode_cols`, ya que `OneHotEncoder` evita introducir orden implícito en variables no ordinales.
* **Resultado del Análisis**
  * Se imprime una lista diferenciada de columnas según el tipo de codificación recomendado. Esto facilita el diseño posterior del pipeline de preprocesamiento, asegurando un tratamiento correcto de las variables categóricas en el modelo.

Este paso es crucial para garantizar que el modelo reciba entradas numéricas bien estructuradas, evitando sesgos implícitos y mejorando su rendimiento general.

In [ ]:
# Aplicar LabelEncoder a las columnas binarias
for col in label_encode_cols:
    X[col] = LabelEncoder().fit_transform(X[col])

# Aplicar OneHotEncoder a las columnas con más de 2 categorías
encoder = OneHotEncoder(drop='first', sparse_output=True)
X_encoded = encoder.fit_transform(X[onehot_encode_cols])

# Combinar las características numéricas y categóricas codificadas
X_rest = X.drop(columns=onehot_encode_cols)
X_sparse = hstack([X_rest, X_encoded])

# División en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_sparse, y, test_size=0.2, random_state=42)

🔹Codificación de Variables Categóricas y Preparación Final de Datos

* **Codificación con LabelEncoder**
  * Se aplica a columnas binarias (`label_encode_cols`) para transformarlas directamente en valores numéricos (`0` y `1`), manteniendo la estructura original sin expansión dimensional.
* **Codificación con OneHotEncoder**
  * Para columnas con más de dos categorías (`onehot_encode_cols`), se utiliza `OneHotEncoder` con `drop='first'` para evitar multicolinealidad.
  * La opción `sparse_output=True` genera una matriz dispersa eficiente en memoria.
* **Integración del Dataset Codificado**
  * `X_rest` conserva las columnas no categóricas.
  * La matriz final `X_sparse` se obtiene al combinar `X_rest` con la matriz codificada `X_encoded` mediante `hstack`, integrando variables categóricas y numéricas en una estructura apta para el modelado.
* **División de Datos**
 * Los datos se dividen en conjuntos de entrenamiento y prueba (`X_train`, `X_test`, `y_train`, `y_test`) mediante `train_test_split`, reservando el 20% para validación (`test_size=0.2`) y asegurando reproducibilidad con `random_state=42`.

In [ ]:
# Imputación de NaN con la más frecuente de cada columna
imputer = SimpleImputer(strategy="most_frequent")
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)


🔹Imputación de Valores Faltantes con SimpleImputer
* **Definición del Imputador**
  * Se utiliza `SimpleImputer(strategy="most_frequent")` para reemplazar los valores nulos (`NaN`) por el valor más frecuente de cada columna. Esta estrategia es útil para variables categóricas o numéricas discretas con una moda dominante.
* **Aplicación en el Conjunto de Entrenamiento**
  * `fit_transform(X_train)` ajusta el imputador aprendiendo el valor más frecuente de cada columna y lo aplica inmediatamente sobre `X_train`, garantizando consistencia interna.
* **Aplicación en el Conjunto de Prueba**
  * `transform(X_test)` aplica la imputación aprendida desde `X_train` a `X_test`, asegurando que no se introduzca información del conjunto de prueba durante el entrenamiento (principio de no-leakage).

In [ ]:
# Entrenar un modelo de Regresión Lineal
linear_model = LinearRegression()

# Realizar predicciones con validación cruzada
y_pred_cv = cross_val_predict(linear_model, X_train, y_train, cv=4)

# Calcular las métricas en validación cruzada
mse_cv = mean_squared_error(y_train, y_pred_cv)
mae_cv = mean_absolute_error(y_train, y_pred_cv)
rmse_cv = np.sqrt(mse_cv)

print(f"MSE Regresión Lineal: {mse_cv:.4f}")
print(f"MAE Regresión Lineal: {mae_cv:.4f}")
print(f"RMSE Regresión Lineal: {rmse_cv:.4f}")


🔹Evaluación del Modelo de Regresión Lineal (Baseline)

Este modelo fue utilizado únicamente como baseline inicial para establecer un punto de comparación. No se considera adecuado para producción debido a limitaciones en su rendimiento predictivo.

1. **Entrenamiento y Evaluación**
* **Modelo**: `LinearRegression()`
* **Entrenamiento**: Se ajustó sobre los datos `X_train` y `y_train`.
* **Validación cruzada**: Se utilizó cross_val_score con 4 pliegues (`cv=4`), evaluando con la métrica neg_mean_squared_error.
* **Promedio del MSE**: Se calculó invirtiendo el signo de la media para interpretar el error en forma positiva.

2. **Resultados**
* **MSE (Mean Squared Error)**: ~0.31
  * Refleja un error cuadrático promedio bajo en la escala logarítmica del target (log(1 + transacciones)), que va aproximadamente de 0 a 6.
* **MAE (Mean Absolute Error)**: ~0.43
  * Indica que la desviación promedio de las predicciones es de 0.43 unidades en la escala logarítmica.
* **RMSE (Root Mean Squared Error)**: ~0.55
  * Proporciona una métrica más intuitiva de error promedio. En este caso, también es bajo, lo que implica que no hay errores extremos frecuentes.

3. **Conclusión**
Aunque los errores obtenidos son bajos en términos absolutos, la regresión lineal no captura relaciones no lineales ni interacciones complejas entre variables. Por ello, aunque sirve como referencia inicial, se requiere un modelo más robusto para mejorar la capacidad predictiva y explotar mejor la estructura del dataset.

In [ ]:
# Paso 1: Convertir variables categóricas en X a numéricas usando OneHotEncoder
X_encoded = pd.get_dummies(X, drop_first=True)

# Paso 2: Concatenar X_encoded e y en un único DataFrame para el cálculo de la correlación
if 'target' in X_encoded.columns:
    X_encoded = X_encoded.drop(columns='target')
df_combined = pd.concat([X_encoded, y.rename("target")], axis=1)

# Paso 3: Calcular la matriz de correlación en el DataFrame combinado
correlation_matrix = df_combined.corr()

# Paso 4: Extraer la fila de correlación del target y ordenarla usando 'by'
try:
    # Filtramos las correlaciones con respecto al 'target', convirtiéndolas en un DataFrame si es necesario
    target_correlation = correlation_matrix[['target']].drop(index='target').sort_values(by='target', ascending=False)

    # Mostrar las 10 características con mayor correlación con el target
    print("Top 10 features más correlacionados con el target:")
    print(target_correlation.head(10))
except KeyError:
    print("Error: La columna 'target' no se encuentra en correlation_matrix.")

🔹Selección del Target para Modelos de Clasificación

* **Target elegido**: Probabilidad de Retención de Clientes
  * **Definición**: Clasificación binaria que distingue entre clientes con alta probabilidad de abandono y aquellos con baja probabilidad de abandono.
    *  Esta variable se puede construir a partir de indicadores como frecuencia de transacciones, antigüedad, uso de productos financieros y niveles de actividad reciente.
  * **Justificación de Negocio**
    * Este target es altamente accionable. Permite a Dormammu identificar clientes en riesgo de fuga y diseñar estrategias de retención proactiva. Al anticipar la pérdida de clientes, el banco puede intervenir con ofertas personalizadas, campañas de fidelización o ajustes en productos, mejorando la rentabilidad a largo plazo y reduciendo los costos de adquisición de nuevos clientes.

In [115]:
# Filtrar solo columnas numéricas para escalamiento
num_features = X.select_dtypes(include=[np.number]).columns
X[num_features] = StandardScaler().fit_transform(X[num_features])

In [ ]:
# Codificación One-Hot de variables categóricas con más de dos categorías
encoder = OneHotEncoder(drop='first', sparse_output=False)
categorical_features = ['Region', 'IndRev_T01']
X_encoded = encoder.fit_transform(X[categorical_features])

# Combinar las características codificadas con el resto del DataFrame
X_rest = X.drop(columns=categorical_features)
X_final = pd.concat([pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(categorical_features)), X_rest.reset_index(drop=True)], axis=1)

# Verificación del resultado
print("Primeras filas del DataFrame codificado:")
print(X_final.head())

🔹Transformación de Atributos para Modelos de Clasificación

Para adaptar el conjunto de datos a modelos de clasificación, se realizó la transformación de variables categóricas mediante codificación One-Hot, asegurando que todas las columnas estén en formato numérico y apropiadas para aprendizaje supervisado.

1. **Codificación One-Hot**
* **Objetivo**: Convertir variables categóricas como `Region` o `IndRev_T01` en un formato binario interpretable por modelos de machine learning.
* **Método**: Cada categoría única se transforma en una columna binaria (`0` o `1`). Se utilizó `drop='first'` para evitar colinealidad entre variables generadas.
* **Ejemplo de columnas resultantes**: `Region_2.0`, `Region_3.0`, etc., indican presencia de cada categoría.

2. **Integración con el Dataset**
* Las variables codificadas se combinaron con las variables numéricas originales y derivadas para conformar el conjunto final (`X_final`).
* El resultado es una matriz de características puramente numérica, lista para alimentar modelos de clasificación como regresión logística, árboles de decisión, o modelos basados en boosting.

3. **Inspección del Resultado**
* Las primeras filas de `X_final` muestran la correcta conversión de variables categóricas y la conservación de variables numéricas clave como Edad, Renta, Antigüedad, y diversos indicadores de actividad financiera.
* El dataset resultante ofrece una representación estructurada y compatible con técnicas de modelado modernas.

In [ ]:
# Visualización de la correlación entre características seleccionadas

# Selección de características específicas para la matriz de correlación
selected_features = ['Edad', 'Renta', 'Antiguedad', 'CUPO_L1', 'CUPO_L2', 'CUPO_MX']

# Generar la matriz de correlación solo para las características seleccionadas
correlation_matrix = X[selected_features].corr()

# Visualizar la matriz de correlación
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", cbar_kws={'label': 'Correlación'})
plt.title("Matriz de Correlación de Características Numéricas ")
plt.show()

🔹Análisis de Correlación entre Variables Numéricas

La matriz de correlación presentada analiza la relación lineal entre seis variables clave: `Edad`, `Renta`, `Antigüedad`, `CUPO_L1`, `CUPO_L2` y `CUPO_MX`. Los coeficientes de correlación oscilan entre -1 y 1, donde:

* +1: Correlación positiva perfecta.
* -1: Correlación negativa perfecta.
* 0: Ausencia de correlación lineal.

Principales Hallazgos
* **`CUPO_L1` vs. `CUPO_MX` (r = 0.66)**
  * Existe una correlación moderadamente alta entre el límite de compras nacionales (`CUPO_L1`) y el cupo internacional (`CUPO_MX`), lo que indica que ambos límites suelen escalar juntos, probablemente debido a políticas crediticias internas.
* **`CUPO_L1` vs. `Antigüedad` (r = 0.40)**
  * Los clientes con mayor antigüedad tienden a tener un límite de crédito más alto, lo cual es consistente con prácticas financieras donde la estabilidad en el tiempo es un factor de confianza.
* **`Edad` vs. `Antigüedad` (r = 0.36) y `CUPO_L1` (r = 0.26)**
  * La edad tiene una correlación moderada con la antigüedad y leve con los límites de crédito, sugiriendo cierta influencia del perfil etario, aunque no es un predictor dominante.
* **`Renta` vs. `Límites de Crédito` (r ≈ 0.25)**
  * A pesar de ser una variable financiera clave, la renta muestra correlaciones relativamente bajas con los cupos de crédito. Esto podría indicar que el otorgamiento de cupo considera otros factores como historial de pago o segmentación comercial.
* **Ausencia de Multicolinealidad Crítica**
  * No se detectan correlaciones extremas (por encima de 0.8 o por debajo de -0.8), lo cual es positivo para el modelado. Las variables aportan información diferenciada y no redundante, reduciendo el riesgo de multicolinealidad.


### Random Forest + Pipeline

In [ ]:
# Resumen de valores nulos antes del preprocesamiento
print("Resumen de valores nulos antes del preprocesamiento:")
print(X.isnull().sum())

🔹Revisión de Completitud de Datos

Tras ejecutar una revisión posterior de los valores nulos en el conjunto de datos, se confirma que todas las columnas están completamente pobladas. Variables clave como `Id`, `Subsegmento`, `Sexo`, `Region`, `Edad`, `UsoL1_T01`, `IndRev_T01` y `target` no presentan valores faltantes.

In [ ]:
# Definir las características numéricas y categóricas (modifica según tus variables)
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Preprocesamiento para características numéricas (imputación y escalado)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

# Preprocesamiento para características categóricas (imputación y codificación)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Crear el preprocesador usando ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Verificar y transformar `y` en una variable binaria si es continuo
if y.dtype == 'float' or y.dtype == 'int':
    threshold = np.median(y)
    y = np.where(y >= threshold, 1, 0)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el pipeline con el preprocesador y el modelo
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', LogisticRegression(max_iter=1000))])

In [ ]:
# Entrenamiento del modelo
pipeline.fit(X_train, y_train)

# Evaluación
y_pred = pipeline.predict(X_test)
print("Evaluación del modelo (Random Forest + Pipeline):")
print(classification_report(y_test, y_pred, digits=4))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))


In [ ]:
# Guardar el pipeline entero (preprocesamiento + modelo)
joblib.dump(pipeline, 'models/modelo_pipeline_rf.pkl')

In [ ]:
# Cargar modelo
modelo_cargado = joblib.load('models/modelo_pipeline_rf.pkl')

# Predecir directamente con el pipeline
y_pred_cargado = modelo_cargado.predict(X_test)

# Evaluación para verificar consistencia
print("Evaluación del modelo cargado:")
print(classification_report(y_test, y_pred_cargado, digits=4))

🔹Evaluación del Modelo: Exactitud e Implicaciones

* **Exactitud Obtenida**
  * El modelo alcanzó una exactitud del 96%, lo cual representa un rendimiento sobresaliente considerando las variables utilizadas. Este nivel de desempeño sugiere que las características disponibles en el dataset son efectivas para capturar los patrones necesarios para la clasificación.
* **Interpretación y Consideraciones para Mejora**
  * La exclusión de ciertas columnas no afectó negativamente el rendimiento del modelo, lo que indica que dichas variables probablemente no aportaban información crítica en su estado actual.
  * No obstante, si estas columnas contienen información futura o si se reduce el sesgo de disponibilidad (por ejemplo, completando valores faltantes con estrategias robustas), podrían evaluarse nuevamente para determinar si contribuyen a mejorar la capacidad predictiva.
  * En el caso de variables categóricas, una estrategia más sólida que la eliminación podría ser asignar una categoría explícita para valores faltantes (como "Desconocido"), preservando la información estructural del dataset sin introducir ruido estadístico innecesario.

🔹Conclusión

El modelo demostró ser eficaz incluso con un subconjunto reducido de variables, lo cual refuerza su robustez y eficiencia. Sin embargo, se recomienda seguir evaluando el valor agregado de variables omitidas a medida que se disponga de datos más completos.

---

**Reconocer Overfitting y Underfitting**

In [ ]:
# Calcular la exactitud en el conjunto de entrenamiento y en el conjunto de prueba
train_accuracy = accuracy_score(y_train, pipeline.predict(X_train))
test_accuracy = accuracy_score(y_test, pipeline.predict(X_test))

print(f"Exactitud en entrenamiento: {train_accuracy:.2f}")
print(f"Exactitud en prueba: {test_accuracy:.2f}")

# Interpretación de overfitting o underfitting
if train_accuracy > test_accuracy + 0.1:
    print("Posible overfitting detectado: el modelo tiene un desempeño mucho mejor en entrenamiento que en prueba.")
elif test_accuracy > train_accuracy:
    print("Posible underfitting detectado: el modelo no está capturando patrones suficientemente bien.")
else:
    print("Buen ajuste: el modelo tiene un desempeño balanceado entre entrenamiento y prueba.")


In [ ]:
# Calcular la exactitud en el conjunto de entrenamiento y en el conjunto de prueba
train_accuracy = accuracy_score(y_train, pipeline.predict(X_train))
test_accuracy = accuracy_score(y_test, y_pred)

print(f"Exactitud en entrenamiento: {train_accuracy:.2f}")
print(f"Exactitud en prueba: {test_accuracy:.2f}")

🔹Evaluación del Ajuste del Modelo: Overfitting vs. Underfitting

* **Análisis del Desempeño**:
  * Para evaluar el comportamiento del modelo, se comparó su rendimiento en los conjuntos de entrenamiento y prueba utilizando la métrica de exactitud (accuracy):
    * Exactitud en entrenamiento: 97%
    * Exactitud en prueba: 96%

* **Interpretación**:
  * La diferencia marginal entre ambos conjuntos sugiere que el modelo generaliza adecuadamente y no presenta overfitting (sobreajuste).
  * Asimismo, dado que ambos resultados son altos, el modelo tampoco evidencia underfitting (subajuste), es decir, logra captar patrones relevantes sin sobreajustarse a los datos de entrenamiento.

* **Mensajes de Advertencia**:
  * Se emitieron advertencias relacionadas con columnas que no contienen valores observados (como 'Sexo' o 'IndRev_T12'). Estas advertencias se refieren a omisiones automáticas en el proceso de imputación, pero no impactan negativamente el rendimiento del modelo, ya que dichas columnas no contenían información útil o estaban correctamente gestionadas en etapas anteriores del preprocesamiento.

🔹Conclusión

El modelo logra un buen equilibrio entre sesgo y varianza, lo que lo convierte en una base sólida para aplicaciones predictivas con los datos actuales. No obstante, es recomendable seguir monitoreando este equilibrio si se incorporan nuevas variables o si se modifica la estrategia de imputación.

---

**Métricas de Evaluación de Modelos de Clasificación**

In [ ]:
# Reporte de clasificación detallado
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

🔹Evaluación del Modelo: Métricas de Clasificación

Se evaluó el rendimiento del modelo utilizando las métricas estándar: precisión, recall, F1-score y exactitud, aplicadas a cada clase del target.

🔹Resultados principales

| Clase    | Precision | Recall | F1-score | Soporte |
| -------- | --------- | ------ | -------- | ------- |
| 0        | 0.96      | 0.96   | 0.96     | 2716    |
| 1        | 0.96      | 0.96   | 0.96     | 2950    |
| Accuracy |           |        | 0.96     | 5666    |

* **Macro Avg (no ponderado)**: Todas las métricas promedian 0.96, lo que indica un desempeño parejo entre clases.
* **Weighted Avg (ponderado)**: Igualmente 0.96, confirmando que el modelo mantiene alta precisión incluso considerando la proporción de cada clase.

🔹Interpretación

* **Rendimiento equilibrado**: El modelo clasifica con la misma eficacia ambas clases, lo que es ideal para tareas donde ambas categorías tienen valor estratégico.
* *No se requieren ajustes adicionales¨**: Dado el balance en el desempeño y la distribución relativamente pareja del soporte entre clases, no se observan problemas de desbalance ni necesidad de técnicas de reponderación.

In [ ]:
# Generar matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Visualizar la matriz de confusión
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de Confusión")
plt.show()

🔹Análisis de la Matriz de Confusión

|        | Predicho 0 | Predicho 1 |
| ------ | ---------- | ---------- |
| Real 0 | 2608 (VN)  | 108 (FP)   |
| Real 1 | 119 (FN)   | 2831 (VP)  |

Interpretación

* **Precisión elevada en ambas clases**: El modelo clasifica correctamente el 96% de los casos de cada clase, con un buen balance entre Verdaderos Positivos (2831) y Verdaderos Negativos (2608).

* **Errores moderados**:
  * **Falsos Positivos (108)**: Casos de clase 0 incorrectamente clasificados como 1.
  * **Falsos Negativos (119)**: Casos de clase 1 predichos como 0.
  * Estos errores son proporcionales y no generan sesgo significativo en el modelo.
* **Balance de clases**: No se observa una desproporción crítica entre clases (2716 vs. 2950), por lo que no es estrictamente necesario aplicar técnicas de balanceo (como resampling o class weighting). Sin embargo, podrían explorarse en una etapa de refinamiento para optimizar la sensibilidad o reducir falsos negativos, dependiendo del costo de error para el negocio.

### Conclusión Final

Este proyecto demostró un enfoque completo y estructurado para el análisis y modelado de datos reales del sector financiero. A través de un proceso de limpieza, transformación y evaluación de múltiples modelos, se lograron los siguientes resultados clave:

* **Calidad del dataset**: Se logró una base de datos limpia, sin valores nulos, con variables transformadas adecuadamente y codificadas para el modelado.
* **Análisis exploratorio**: Identificó patrones relevantes, como la alta concentración de clientes en ciertos segmentos (Región 13, subsegmento 170), y un uso predominantemente bajo de productos financieros.
* **Modelado de regresión**: El modelo de regresión lineal aplicado al target transformado (log(1 + transacciones anuales)) logró un desempeño aceptable (MSE: 0.31, RMSE: 0.55), aunque se reconoció que no era ideal para producción.
* **Modelado de clasificación**: Se desarrolló un modelo robusto de clasificación binaria para la retención de clientes, alcanzando una exactitud del 96%, sin señales de overfitting ni underfitting. La matriz de confusión y métricas como precisión, recall y F1-score confirmaron un desempeño equilibrado entre clases.